In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline

In [2]:
data_103507 = pd.read_csv('./data/[new] yancheng_train_20171226.csv')


C:\Users\jinda\AppData\Local\Continuum\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (15,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
price_map = {'5WL':0, '5-8W':1, '8-10W':2, '10-15W':3, '15-20W':4, '20-25W':5, '25-35W':6, '35-50W':7, '50-75W':8}
data_103507['price_level'] = data_103507['price_level'].apply(lambda x: price_map[x])
if_charging_map = {'L': 1, 'T': 0}
data_103507['if_charging'] = data_103507['if_charging'].apply(lambda x: if_charging_map[x])
rated_passenger_map = {'5':0, '7-8':1, '7':2, '6-7':3, '6-8':4, '4':5, '4-5':6, '5-7':7, '5-8':8, '9':9}
data_103507['rated_passenger'] = data_103507['rated_passenger'].apply(lambda x: rated_passenger_map[x])


In [4]:
list_all = []
id_all = pd.read_csv('./data/yancheng_testA_20171225.csv')
for f in range(len(id_all)):
    list_all.append(data_103507[id_all.iloc[f].class_id == data_103507.class_id])

In [5]:
for ff in range(len(list_all)):
    list_all[ff] = list_all[ff].sort_values(by='sale_date')


In [6]:
list_merge = []
for single_class_id in list_all:
    last = 0
    list_merge_specific = pd.DataFrame(columns = ["sale_date", "class_id", "brand_id", "compartment", "department_id",
                                                  "if_charging","price_level", "driven_type_id", "newenergy_type_id",
                                                  "emission_standards_id", "if_MPV_id", "if_luxurious_id", "cylinder_number",
                                                  "rated_passenger","sale_quantity"])
    for ff in range(len(single_class_id)):
        now = single_class_id.iloc[ff].sale_date
        if last == now:
            list_merge_specific.iloc[-1].sale_quantity +=  single_class_id.iloc[ff].sale_quantity
        else:
            list_merge_specific.loc[list_merge_specific.shape[0]+1] = {"sale_date":single_class_id.iloc[ff].sale_date,
                                        "class_id":single_class_id.iloc[ff].class_id, 
                                        "brand_id":single_class_id.iloc[ff].brand_id,
                                        "if_charging":single_class_id.iloc[ff].if_charging,
                                        "compartment":single_class_id.iloc[ff].compartment,
                                        "department_id":single_class_id.iloc[ff].department_id,
                                        "price_level":single_class_id.iloc[ff].price_level,
                                        "driven_type_id":single_class_id.iloc[ff].driven_type_id,
                                        "newenergy_type_id":single_class_id.iloc[ff].newenergy_type_id,
                                        "emission_standards_id":single_class_id.iloc[ff].emission_standards_id, 
                                        "if_MPV_id":single_class_id.iloc[ff].if_MPV_id, 
                                        "if_luxurious_id":single_class_id.iloc[ff].if_luxurious_id, 
                                        "cylinder_number":single_class_id.iloc[ff].cylinder_number,
                                        "rated_passenger":single_class_id.iloc[ff].rated_passenger,
                                        "sale_quantity":single_class_id.iloc[ff].sale_quantity}
        last = now
    list_merge.append(list_merge_specific)

In [7]:
list_merge[0].head(5)

,sale_date,class_id,brand_id,compartment,department_id,if_charging,price_level,driven_type_id,newenergy_type_id,emission_standards_id,if_MPV_id,if_luxurious_id,cylinder_number,rated_passenger,sale_quantity
1,201503,103507,831,2,3,1,3,3,1,3,2,1,4,0,58
2,201504,103507,831,2,3,1,3,1,1,3,2,1,4,0,232
3,201505,103507,831,2,3,0,3,1,1,3,2,1,4,0,226
4,201506,103507,831,2,3,1,3,1,1,3,2,1,4,0,286
5,201507,103507,831,2,3,1,3,3,1,1,2,1,4,0,297


In [9]:
train_all = []
for ff in list_merge:
    every = []
    for f in range(1, len(ff)-1):
        if ff.iloc[f].sale_date%1000 - 1 ==  ff.iloc[f-1].sale_date%1000 and ff.iloc[f+1].sale_date%1000 == ff.iloc[f].sale_date%1000 + 1:
            every.append(ff.iloc[f].class_id)
            every.append(((ff.iloc[f].sale_date)%1000)//100)   # 年份
            every.append(((ff.iloc[f].sale_date)%100))  # 月份
            every.append((ff.iloc[f].brand_id))
            every.append(ff.iloc[f].compartment)
            every.append(ff.iloc[f].department_id)
            every.append(ff.iloc[f].price_level)
            every.append(ff.iloc[f].if_charging)
            every.append(ff.iloc[f].driven_type_id)
            every.append(ff.iloc[f].newenergy_type_id)
            every.append(ff.iloc[f].emission_standards_id)
            every.append(ff.iloc[f].if_MPV_id)
            every.append(ff.iloc[f].if_luxurious_id)
            every.append(ff.iloc[f].cylinder_number)
            every.append(ff.iloc[f].rated_passenger)
            every.append(ff.iloc[f-1].sale_quantity)   # 第一个月
            every.append(ff.iloc[f].sale_quantity)     # 第二个月
            every.append(ff.iloc[f+1].sale_quantity)   # 第三个月
            train_all.append(every)
            every = []
train_all[0]

[103507, 5, 4, 831, 2, 3, 3, 1, 1, 1, 3, 2, 1, 4, 0, 58, 232, 226]

<font color=orange size=6>用于再次合并</font>

In [10]:
train_all = pd.DataFrame(np.array(train_all))

In [11]:
train_all.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,103507,5,4,831,2,3,3,1,1,1,3,2,1,4,0,58,232,226
1,103507,5,5,831,2,3,3,0,1,1,3,2,1,4,0,232,226,286
2,103507,5,6,831,2,3,3,1,1,1,3,2,1,4,0,226,286,297
3,103507,5,7,831,2,3,3,1,3,1,1,2,1,4,0,286,297,355
4,103507,5,8,831,2,3,3,1,1,1,3,2,1,4,0,297,355,442


In [12]:
pd.DataFrame.to_csv(train_all, './data/yanchen_data_process_for_train_201801311718.csv', index=False)

<font color=orange size=6>对每个参数进行分析</font>

In [12]:
print(list_merge[0].columns)
print(list_merge[0].head(5))
list_all_df = pd.DataFrame.merge(list_merge[0],list_merge[1])
for ff in list_merge:
    list_all_df = pd.DataFrame.merge(list_all_df, ff, how='outer')

Index(['sale_date', 'class_id', 'brand_id', 'compartment', 'department_id',
       'if_charging', 'price_level', 'driven_type_id', 'newenergy_type_id',
       'emission_standards_id', 'if_MPV_id', 'if_luxurious_id',
       'cylinder_number', 'rated_passenger', 'sale_quantity'],
      dtype='object')
  sale_date class_id brand_id compartment department_id if_charging  \
1    201503   103507      831           2             3           1   
2    201504   103507      831           2             3           1   
3    201505   103507      831           2             3           1   
4    201506   103507      831           2             3           1   
5    201507   103507      831           2             3           1   

  price_level driven_type_id newenergy_type_id emission_standards_id  \
1           3              3                 1                     3   
2           3              1                 1                     3   
3           3              1                 1          

In [16]:
test_all = []
for f in range(len(id_all)):
    every = []
    for ff in list_merge:
        if ff.iloc[0].class_id == id_all.iloc[f].class_id:
            temp = ff.iloc[np.array([ff.class_id == id_all.iloc[f].class_id] and [ff.sale_date == 201710]).tolist()[0].index(True)]
            print(ff.class_id)
            every.append(temp.class_id)
            every.append(((temp.sale_date)%1000)//100)   # 年份
            every.append(((temp.sale_date)%100))  # 月份
            every.append((temp.brand_id))
            every.append(temp.compartment)
            every.append(temp.department_id)
            every.append(temp.price_level)
            every.append(temp.if_charging)
            every.append(temp.driven_type_id)
            every.append(temp.newenergy_type_id)
            every.append(temp.emission_standards_id)
            every.append(temp.if_MPV_id)
            every.append(temp.if_luxurious_id)
            every.append(temp.cylinder_number)
            every.append(temp.rated_passenger)
            every.append(ff.iloc[np.array([ff.class_id == id_all.iloc[f].class_id] and [ff.sale_date == 201710]).tolist()[0].index(True)-1].sale_quantity)   # 第一个月
            every.append(temp.sale_quantity)     # 第二个月
            test_all.append(every)
            every = []
    print("ok")
        

1     103507
2     103507
3     103507
4     103507
5     103507
6     103507
7     103507
8     103507
9     103507
10    103507
11    103507
12    103507
13    103507
14    103507
15    103507
16    103507
17    103507
18    103507
19    103507
20    103507
21    103507
22    103507
23    103507
24    103507
25    103507
26    103507
27    103507
28    103507
29    103507
30    103507
31    103507
32    103507
Name: class_id, dtype: object
ok
1     124140
2     124140
3     124140
4     124140
5     124140
6     124140
7     124140
8     124140
9     124140
10    124140
11    124140
12    124140
13    124140
14    124140
15    124140
16    124140
17    124140
18    124140
19    124140
20    124140
21    124140
22    124140
23    124140
24    124140
25    124140
26    124140
27    124140
28    124140
29    124140
30    124140
31    124140
32    124140
33    124140
34    124140
35    124140
36    124140
37    124140
38    124140
39    124140
40    124140
41    124140
42    124140
Name:

ok
1     249875
2     249875
3     249875
4     249875
5     249875
6     249875
7     249875
8     249875
9     249875
10    249875
11    249875
12    249875
Name: class_id, dtype: object
ok
1     250658
2     250658
3     250658
4     250658
5     250658
6     250658
7     250658
8     250658
9     250658
10    250658
11    250658
12    250658
13    250658
14    250658
15    250658
16    250658
17    250658
18    250658
19    250658
20    250658
Name: class_id, dtype: object
ok
1     265980
2     265980
3     265980
4     265980
5     265980
6     265980
7     265980
8     265980
9     265980
10    265980
11    265980
12    265980
13    265980
14    265980
15    265980
16    265980
17    265980
18    265980
19    265980
20    265980
21    265980
22    265980
23    265980
24    265980
25    265980
26    265980
27    265980
28    265980
29    265980
30    265980
       ...  
40    265980
41    265980
42    265980
43    265980
44    265980
45    265980
46    265980
47    265980
48    26

ok
1     321683
2     321683
3     321683
4     321683
5     321683
6     321683
7     321683
8     321683
9     321683
10    321683
11    321683
12    321683
13    321683
14    321683
15    321683
16    321683
17    321683
18    321683
19    321683
20    321683
21    321683
22    321683
23    321683
24    321683
25    321683
26    321683
27    321683
28    321683
29    321683
30    321683
       ...  
41    321683
42    321683
43    321683
44    321683
45    321683
46    321683
47    321683
48    321683
49    321683
50    321683
51    321683
52    321683
53    321683
54    321683
55    321683
56    321683
57    321683
58    321683
59    321683
60    321683
61    321683
62    321683
63    321683
64    321683
65    321683
66    321683
67    321683
68    321683
69    321683
70    321683
Name: class_id, Length: 70, dtype: object
ok
1    340175
2    340175
3    340175
4    340175
Name: class_id, dtype: object
ok
1     347384
2     347384
3     347384
4     347384
5     347384
6     347384


ok
1     395234
2     395234
3     395234
4     395234
5     395234
6     395234
7     395234
8     395234
9     395234
10    395234
11    395234
12    395234
13    395234
14    395234
15    395234
16    395234
17    395234
18    395234
19    395234
20    395234
21    395234
22    395234
23    395234
24    395234
25    395234
26    395234
27    395234
28    395234
29    395234
30    395234
       ...  
33    395234
34    395234
35    395234
36    395234
37    395234
38    395234
39    395234
40    395234
41    395234
42    395234
43    395234
44    395234
45    395234
46    395234
47    395234
48    395234
49    395234
50    395234
51    395234
52    395234
53    395234
54    395234
55    395234
56    395234
57    395234
58    395234
59    395234
60    395234
61    395234
62    395234
Name: class_id, Length: 62, dtype: object
ok
1     401052
2     401052
3     401052
4     401052
5     401052
6     401052
7     401052
8     401052
9     401052
10    401052
11    401052
12    401052
13 

ok
1     496459
2     496459
3     496459
4     496459
5     496459
6     496459
7     496459
8     496459
9     496459
10    496459
11    496459
12    496459
13    496459
14    496459
15    496459
16    496459
17    496459
18    496459
19    496459
20    496459
21    496459
22    496459
23    496459
24    496459
25    496459
26    496459
27    496459
28    496459
29    496459
30    496459
31    496459
32    496459
33    496459
34    496459
35    496459
36    496459
37    496459
38    496459
39    496459
40    496459
41    496459
42    496459
Name: class_id, dtype: object
ok
1     501564
2     501564
3     501564
4     501564
5     501564
6     501564
7     501564
8     501564
9     501564
10    501564
11    501564
12    501564
Name: class_id, dtype: object
ok
1     510309
2     510309
3     510309
4     510309
5     510309
6     510309
7     510309
8     510309
9     510309
10    510309
11    510309
12    510309
13    510309
14    510309
15    510309
16    510309
17    510309
18    51

ok
1    612523
2    612523
3    612523
4    612523
Name: class_id, dtype: object
ok
1     614059
2     614059
3     614059
4     614059
5     614059
6     614059
7     614059
8     614059
9     614059
10    614059
11    614059
12    614059
13    614059
14    614059
15    614059
16    614059
17    614059
18    614059
19    614059
20    614059
21    614059
22    614059
23    614059
24    614059
25    614059
26    614059
27    614059
28    614059
29    614059
30    614059
       ...  
41    614059
42    614059
43    614059
44    614059
45    614059
46    614059
47    614059
48    614059
49    614059
50    614059
51    614059
52    614059
53    614059
54    614059
55    614059
56    614059
57    614059
58    614059
59    614059
60    614059
61    614059
62    614059
63    614059
64    614059
65    614059
66    614059
67    614059
68    614059
69    614059
70    614059
Name: class_id, Length: 70, dtype: object
ok
1     619401
2     619401
3     619401
4     619401
5     619401
6     619401


1    714150
2    714150
3    714150
4    714150
5    714150
6    714150
7    714150
Name: class_id, dtype: object
ok
1     714152
2     714152
3     714152
4     714152
5     714152
6     714152
7     714152
8     714152
9     714152
10    714152
11    714152
12    714152
13    714152
14    714152
15    714152
16    714152
17    714152
18    714152
19    714152
20    714152
21    714152
22    714152
23    714152
24    714152
25    714152
26    714152
27    714152
28    714152
29    714152
30    714152
       ...  
41    714152
42    714152
43    714152
44    714152
45    714152
46    714152
47    714152
48    714152
49    714152
50    714152
51    714152
52    714152
53    714152
54    714152
55    714152
56    714152
57    714152
58    714152
59    714152
60    714152
61    714152
62    714152
63    714152
64    714152
65    714152
66    714152
67    714152
68    714152
69    714152
70    714152
Name: class_id, Length: 70, dtype: object
ok
1     714860
2     714860
3     714860
4     

ok
1     851857
2     851857
3     851857
4     851857
5     851857
6     851857
7     851857
8     851857
9     851857
10    851857
11    851857
12    851857
13    851857
14    851857
15    851857
16    851857
17    851857
18    851857
19    851857
Name: class_id, dtype: object
ok
1     854079
2     854079
3     854079
4     854079
5     854079
6     854079
7     854079
8     854079
9     854079
10    854079
11    854079
12    854079
13    854079
14    854079
15    854079
16    854079
17    854079
18    854079
19    854079
20    854079
21    854079
22    854079
23    854079
24    854079
25    854079
26    854079
27    854079
28    854079
29    854079
30    854079
31    854079
32    854079
33    854079
34    854079
35    854079
36    854079
37    854079
38    854079
39    854079
40    854079
41    854079
42    854079
43    854079
44    854079
45    854079
46    854079
47    854079
48    854079
49    854079
50    854079
51    854079
52    854079
Name: class_id, dtype: object
ok
1    854

1     973106
2     973106
3     973106
4     973106
5     973106
6     973106
7     973106
8     973106
9     973106
10    973106
11    973106
12    973106
Name: class_id, dtype: object
ok
1     978089
2     978089
3     978089
4     978089
5     978089
6     978089
7     978089
8     978089
9     978089
10    978089
11    978089
12    978089
13    978089
14    978089
15    978089
16    978089
17    978089
18    978089
19    978089
20    978089
21    978089
22    978089
23    978089
24    978089
25    978089
26    978089
27    978089
28    978089
29    978089
30    978089
31    978089
32    978089
33    978089
34    978089
35    978089
36    978089
37    978089
38    978089
39    978089
Name: class_id, dtype: object
ok


<font color=green size=4>list_merge -> list, id_all -> DateFrame</font>

In [17]:
test_all_df = pd.DataFrame(test_all)
test_all

[[103507, 7, 10, 831, 2, 3, 3, 1, 1, 1, 1, 2, 1, 4, 0, 186, 180],
 [124140, 7, 10, 783, 2, 1, 3, 0, 1, 1, 1, 2, 1, 4, 0, 285, 263],
 [125403, 7, 10, 761, 2, 2, 7, 1, 1, 1, 1, 2, 1, 4, 2, 328, 156],
 [136916, 7, 10, 106, 3, 5, 4, 1, 1, 1, 1, 2, 1, 4, 0, 145, 162],
 [169673, 7, 10, 682, 2, 1, 1, 0, 1, 1, 1, 2, 1, 4, 2, 219, 152],
 [175962, 7, 10, 750, 3, 4, 3, 1, 1, 1, 1, 2, 1, 4, 0, 214, 237],
 [178529, 7, 10, 98, 1, 1, 0, 1, 2, 1, 1, 2, 1, 4, 2, 165, 148],
 [186250, 7, 10, 761, 2, 2, 3, 1, 1, 1, 1, 2, 1, 4, 0, 83, 83],
 [194201, 7, 10, 831, 2, 3, 2, 1, 1, 1, 1, 2, 1, 4, 0, 239, 378],
 [194450, 7, 10, 836, 2, 3, 4, 0, 1, 1, 1, 2, 1, 4, 0, 307, 271],
 [198427, 7, 10, 12, 3, 1, 1, 1, 1, 1, 1, 2, 1, 4, 0, 125, 99],
 [206765, 7, 10, 761, 3, 2, 4, 1, 1, 1, 1, 2, 1, 4, 0, 2347, 2057],
 [209945, 7, 10, 814, 3, 4, 7, 0, 2, 1, 1, 2, 2, 4, 0, 273, 91],
 [219195, 7, 10, 638, 2, 1, 3, 0, 1, 1, 1, 2, 1, 4, 0, 160, 125],
 [221795, 7, 10, 872, 2, 4, 6, 0, 1, 1, 1, 2, 2, 4, 0, 337, 391],
 [245609, 7, 1

In [18]:
pd.DataFrame.to_csv(test_all_df,'./data/yanchen_data_process_for_test_201801311836.csv',index=False)